In [1]:
from arcgis.gis import GIS
gis = GIS(url="https://igestrategies.maps.arcgis.com", username="Piphi5")

Enter password:  ·········


In [27]:
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from arcgis.features import FeatureLayer, GeoAccessor

    
groups_url = "https://services4.arcgis.com/feSRYIeirl5nUF92/arcgis/rest/services/MHM_Group_Data/FeatureServer/0"
mhm_url = "https://services4.arcgis.com/feSRYIeirl5nUF92/arcgis/rest/services/GLOBE_Observer_MHM_Measurements_Updated_Daily_Public_View/FeatureServer/0"

def get_data():
    def get_df(url):
        temp_layer = FeatureLayer(url)
        temp_df = GeoAccessor.from_layer(temp_layer)
        temp_df = temp_df.drop(["SHAPE", "ObjectId"], 1)
        return temp_df
    
    df = get_df(mhm_url)
    groups_df = get_df(groups_url)
    sorted_groups_df = groups_df.sort_values("measuredDate", ascending=False).reset_index()
    latest_date = sorted_groups_df["measuredDate"].loc[0]
    df = df[df["measuredDate"] > latest_date].append(groups_df, ignore_index=True)
    return df

data = None
removed_groups = []

In [28]:
df = get_data()

In [29]:
import os
if not os.path.exists("Images"):
    os.makedirs("Images")

In [30]:
import geopandas as gpd
from arcgis.features import GeoAccessor
from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry

groups_features = pd.DataFrame()
def remove_groups():
    global data, removed_groups
    for removed_group in removed_groups:
        data = data[data["Group Data"] != removed_group]
        
def process_data(threshold):
    global df, data, removed_groups, groups_features
    suspect_df = df.groupby(by=['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude']).filter(lambda x: len(x) > threshold)

    # identify groups
    suspect_groups = suspect_df.groupby(by = ['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude'])

    # create a groups dataset
    groups_data = pd.DataFrame()
    group_num = 1 
    for group, temp in suspect_groups:
        new_df = temp.copy(True)
        new_df["Group Data"] = str(group)
        new_df["Group Name"] = str(group_num)

        group_num += 1
        groups_data = groups_data.append(new_df, ignore_index = True)

    data = GeoAccessor.from_xy(groups_data, "longitude", "latitude")
    remove_groups()
    groups_features = data.copy(True)

    groups_features.mosquitohabitatmapperComments.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
process_data(10)

In [31]:
from arcgis.features import FeatureLayerCollection
from IPython.display import display, clear_output
import ipywidgets as widgets



m1 = gis.map('MHM Groups')


def add_layer(map_obj):
    map_obj.basemap = 'dark-gray-vector'
    try:
        global groups_features
        groups_features.spatial.plot(map_widget=map_obj, renderer_type='s', colors='Reds_r', outline_color='Reds', marker_size=10)

    except KeyError:
        print("no such group exists")
    
def update_map():
    global m1
    m1.remove_layers()
    add_layer(m1)

In [32]:
group_name = ""
group_data = pd.DataFrame()
def move_to_group(group):
    global group_data, m1
    if group != "None":
        group_data = data[data["Group Name"] == group]
        latitude = group_data["latitude"].iloc[0]
        longitude = group_data["longitude"].iloc[0]
        m1.center = [latitude, longitude]
    else:
        group_data = pd.DataFrame()
    
       
def select_group(group):
    global group_name, group_data, selected_group
    group_name = group
    move_to_group(group)
    display(group_data)


In [33]:
stored_threshold = 10
threshold_output = widgets.Output()
deleted_length = len(removed_groups)

@threshold_output.capture(clear_output=True, wait=True)
def get_groups(threshold):
    global stored_threshold, deleted_length
    if stored_threshold != threshold or deleted_length != len(removed_groups):
        stored_threshold = threshold
        deleted_length = len(removed_groups)
        process_data(stored_threshold)
        update_map()
    options = data["Group Name"].unique().tolist()
    options.insert(0, "None")
    display(interactive(select_group, group = options))
    


threshold_modifier = widgets.VBox([interactive(get_groups, threshold = widgets.IntSlider(10, 1, 100, continuous_update = False)), threshold_output])

In [34]:
import OverwriteFS
def download_groups(_):
    with download_out:
        global stored_threshold, data
        filename = f"MHM_Groups.csv"
        item_id = "3eff230387c045f79a70e40195961452"
        item_name = "MHM Group Data"
        
        item = gis.content.get(item_id)
        
        print(f"Saving Groups CSV as {filename}")
        data.to_csv(filename)
        outcome = OverwriteFS.overwriteFeatureService(item, updateFile=filename, touchItems=True, verbose=True)
        print(f"Finished saving file: {outcome}")
    
download_button = widgets.Button(description='Download Data')
download_out = widgets.Output()

# linking button and function together using a button's method
download_button.on_click(download_groups)

# create download widget
download_button = widgets.VBox([download_button, download_out])

In [64]:
import glob
import re
import requests
import sys
from PIL import Image


def show_images(file):
    try:
        display(Image("Images/" + file))
    except TypeError:
        print("No Pictures to download")

def download_picture(url, directory):
  
    def get_picture(file_name):
        downloaded_obj = requests.get(url, allow_redirects=True)
        parent_dir = os.path.join(directory, file_name)
        with open(parent_dir, "wb") as file:
            file.write(downloaded_obj.content)
    if "https://" in url:
        photo_id = re.search(r'(?<=\d\d\d\d\/\d\d\/\d\d\/).*(?=\/)', url).group(0)
        print(url)
        file_name = f"{photo_id}.jpg".replace(":","-")
        get_picture(file_name)

        
def get_pictures(group):
    files = glob.glob('Images/*')
    for f in files:
        os.remove(f)

    group_water_source = group["mosquitohabitatmapperWaterSou_1"].dropna().tolist()
    group_abdomen = group["mosquitohabitatmapperAbdomenClo"].dropna().tolist()
    group_larvae = group["mosquitohabitatmapperLarvaFullB"].dropna().tolist()
    group_urls = group_water_source + group_abdomen + group_larvae
    dl = 0
    total_length = len(group_urls)
    for urls in group_urls:
        if urls:
            dl += 1
            urls = urls.split(";")
            for url in urls:
                download_picture(url, "Images")
        
        done = int(50 * dl / total_length)
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
        sys.stdout.flush()

def download_photos(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        print("Downloading Photos:")
        get_pictures(data[data["Group Name"] == group_name])
        print(' Finished Downloading!')
        
        photo_displayer = interactive(show_images, file=os.listdir('Images/')) 
        display(photo_displayer)

In [65]:
button = widgets.Button(description='Get Photos')
out = widgets.Output()

# linking button and function together using a button's method
button.on_click(download_photos)
# displaying button and its output together
button = widgets.VBox([button,out])

In [66]:
def update_ui():
    global stored_threshold, threshold_modifier
    remove_groups()
    display(threshold_modifier)
    
def delete_group(_):
    global removed_groups, group_data
    removed_groups.append(group_data["Group Data"].iloc[0])
    update_ui()
    
def undo_remove(_):
    global removed_groups
    removed_groups.pop()
    update_ui()
    
def update_local(_):
    global df, stored_threshold
    df = get_data()
    process_data(stored_threshold)
    update_ui()
    
    

In [67]:
delete_button = widgets.Button(description = "Delete Group")
delete_button.on_click(delete_group)

update_button = widgets.Button(description = "Update From Sever")
update_button.on_click(update_local)

undo_button = widgets.Button(description = "Undo Deletion")
undo_button.on_click(undo_remove)

data_flagging = widgets.HBox([delete_button, undo_button, update_button])
data_interface = widgets.VBox([data_flagging, threshold_modifier])

In [68]:
tabs = widgets.Tab(children = [data_interface, button, download_button])

tabs.set_title(0, "Threshold Selection")
tabs.set_title(1, "Photo Downloader")
tabs.set_title(2, "Data Download")

In [69]:
add_layer(m1)
m1

MapView(jupyter_target='lab', layout=Layout(height='400px', width='100%'), ready=True)

In [70]:
tabs